In [191]:
import advent

data = advent.get_char_grid(23, 'txt')

In [192]:
def walkable(node: tuple[int, int]) -> bool:
    return node[0] >= 0 and node[0] < data.shape[0] and node[1] >= 0 and node[1] < data.shape[1] and data[node] != '#'

def adjacent(node: tuple[int, int, int, int]) -> list[tuple[tuple[int, int, int, int], int]]:
    # To prevent stepping back and forth, a node is 4 numbers: current and last
    px, py, x, y = node
    result: list[tuple[tuple[int, int, int, int], int]] = []
    if walkable((x-1, y)) and data[x-1, y] != 'v' and (x-1, y) != (px, py): result.append(((x, y, x-1, y), -1))
    if walkable((x+1, y)) and data[x+1, y] != '^' and (x+1, y) != (px, py): result.append(((x, y, x+1, y), -1))
    if walkable((x, y-1)) and data[x, y-1] != '>' and (x, y-1) != (px, py): result.append(((x, y, x, y-1), -1))
    if walkable((x, y+1)) and data[x, y+1] != '<' and (x, y+1) != (px, py): result.append(((x, y, x, y+1), -1))
    return result

In [193]:
# Unfortunately, cannot do this with dijkstra/heapq
# It can be done similarly to dijkstra, by just brute forcing it
# that is, remove all the optimizations and reasons why dijkstra is fast
from typing import Any, Callable
from tqdm.notebook import tqdm

Node = Any

def solve_maze(start: Node, adjacent: Callable[[Node], list[tuple[Node, int]]]):
    global longest
    longest = 0
    f: list[tuple[int, Node]]= [(0, start)] # [(g, n)]
    open: dict[Node, int] = {} # track longest routes so far
    # We have no closed because we are not doing dijkstra
    while f:
        current_g, current_n = f.pop(0)
        for adj, adj_g in adjacent(current_n):
            new_g = current_g + adj_g
            # Only add the adjacent node if we don't already have a longer path found
            if adj not in open or open[adj] > new_g:
                # We found a new longest route
                f.append((new_g, adj))
                open[adj] = new_g
    return open

In [194]:
# Part 1. we just brute force it, keep track of the previous node
# to prevent stepping back-and-forth in place

start = (0, 0, 0, 1)
open = solve_maze(start, adjacent)
for x in open:
    if x[2] == data.shape[0]-1: # The ending node
        print(open[x] * -1)

2386


In [144]:
# Part 2, attempt 1
# Again brute force, but instead of only the previous,
# We also keep a list of intersections we've visited
# unfortunately, doesn't work fast enough

# Path is the previous, as well as a list of intersections
Point = tuple[int, int]
Path = tuple[Point, Point, frozenset[Point]]

# The problem with doing a 'naive approach' where you just store the entire
# path and check if the node is in the path: its too slow :(
# Instead, let's only store the intersections

def is_intersection(x: int, y:int):
    if x == 0 or x == data.shape[0] - 1: return True
    return ((data[x-1, y] == '#') + \
            (data[x+1, y] == '#') + \
            (data[x, y-1] == '#') + \
            (data[x, y+1] == '#')) < 2

# Add x, y to node to create a new Node. Assume x, y is walkable
# If x, y is an already visited intersection, return None
def create_new_node(node: Path, x: int, y: int, prev: Point) -> Path|None:
    if is_intersection(x, y):
        if (x, y) in node[2]: return None
        new_set = node[2].union([(x, y)])
        return (prev, (x, y), new_set)
    else: return (prev, (x, y), node[2])

def adjacent_path(node: Path) -> list[tuple[Path, int]]:
    x, y = node[1]
    result: list[tuple[Path, int]] = []
    if walkable((x-1, y)) and (x-1, y) != node[0]:
        newnode = create_new_node(node, x-1, y, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x+1, y)) and (x+1, y) != node[0]:
        newnode = create_new_node(node, x+1, y, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x, y-1)) and (x, y-1) != node[0]:
        newnode = create_new_node(node, x, y-1, (x, y))
        if newnode: result.append((newnode, -1))
    if walkable((x, y+1)) and (x, y+1) != node[0]:
        newnode = create_new_node(node, x, y+1, (x, y))
        if newnode: result.append((newnode, -1))
    return result

start: Path = ((0, 1), (0, 1), frozenset([(0, 1)]))
open = solve_maze(start, adjacent_path)

In [145]:
# Part 2, attempt 2
# Try to 'speed up' by immediately following the path until the next intersection
# to prevent redundantly walking the path over and over again
# still too slow

def tadd(a: tuple[int, int], b: tuple[int, int]):
    return a[0] + b[0], a[1] + b[1]

def follow_path(point: Point, prev: Point) -> tuple[Point, int]:
    # Returns the next intersection and the length of the path
    length = 1
    while not is_intersection(point[0], point[1]):
        length += 1
        for dir in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            new = tadd(point, dir)
            if new != prev and walkable(new):
                prev, point = point, new
                break
    return point, length

# NewPath is just our current location and a set of locations we've visited before
# every point we visit is an intersection, to save processing time
NewPath = tuple[Point, frozenset[Point]]

def adjacent_follow(node: NewPath):
    result: list[tuple[NewPath, int]] = []
    for dir in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        point = tadd(node[0], dir)
        if walkable(point):
            newpoint, cost = follow_path(point, node[0])
            if newpoint not in node[1]:
                newnode = (newpoint, node[1].union([newpoint]))
                result.append((newnode, -1 * cost))
    return result

start: NewPath = ((0, 1), frozenset([(0, 1)]))
# too slow
#open = solve_maze(start, adjacent_follow)

[(((5, 7), frozenset({(5, 7)})), -19)]

In [184]:
# Part 2, attempt 3

# I counted there were about 34 intersections, so I try to form a graph with 34 elements
# This is basically the same maze algo as attempt 1 and 2 but with even more precomputation

nodes: list[Point] = [(0, 1)]
f: list[Point] = [(0, 1)]
adjacents: dict[Point, list[tuple[Point, int]]] = {}
edges: dict[tuple[Point, Point], int] = {}

while len(f) > 0:
    node = f.pop()
    new_adjacents: list[tuple[Point, int]] = []
    for dir in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        point = tadd(node, dir)
        if walkable(point):
            newpoint, cost = follow_path(point, node)
            if newpoint not in nodes: # We dont have it yet, add it
                nodes.append(newpoint)
                f.append(newpoint)
            new_adjacents.append((newpoint, -1 * cost))
            edges[(node, newpoint)] = -1 * cost
    adjacents[node] = new_adjacents

def adjacent_graph(node: NewPath) -> list[tuple[NewPath, int]]:
    next =  adjacents[node[0]]
    result: list[tuple[NewPath, int]] = []
    for n, score in next:
        if n in node[1]: continue
        result.append(((n, node[1].union([n])), score))
    return result

start: NewPath = ((0, 1), frozenset([(0, 1)]))
# But still too slow
#open = solve_maze(start, adjacent_graph)

In [204]:
# Part 2, attempt 4
# Try dynamic programming
from functools import cache

# name is dumb, we are actually finding the shortest since weights are negative
@cache
def find_longest_path(start: Point=(0, 1), end: Point=(140, 139), visited: frozenset[Point] = frozenset([])):
    if start == end: return 0
    subresults: list[int] = []
    for adj, cost in adjacents[start]:
        if adj in visited: continue
        new_visited = visited.union([adj])
        subresults.append(cost + find_longest_path(adj, end, new_visited))
    if len(subresults) == 0: return 1_000_000_000 # large number so it will never be longest
    return min(subresults)

find_longest_path()
# Didn't think that would work (since there's no progress bar) but it just finished after 75 seconds!

-6246